In [ ]:
#!pip install git+https://github.com/karask/python-bitcoin-utils.git

# Exercise 1 


## Import

In [ ]:
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
from bitcoinutils.keys import P2pkhAddress, PrivateKey
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script
import requests

## Key and Wallet Creation

In [ ]:
# Testnet environment setup
setup('testnet')

private_key = PrivateKey()
public_key = private_key.get_public_key()
address = public_key.get_address()

print("Private Key (WIF format):", private_key.to_wif())
print("Public Key:", public_key.to_hex())
print("Bitcoin Address (P2PKH):", address.to_string())

Private Key (WIF format): cUYYNWUYYMUQncmo8curJihmDdNXbGAKKWpiCb1MdjkdcSjCGJuC

Public Key: 03db1db7e0d759dd6f87abf8be0fcf3d0464f71d234b2abfba5a12c147d39a06a6

Bitcoin Address (P2PKH): mkvbvcrfDbJC2je27tSyA9MH9rnjPkFRXt

## P2PKH Transaction

### Params

In [ ]:
# My keys
priv_key = PrivateKey('cUYYNWUYYMUQncmo8curJihmDdNXbGAKKWpiCb1MdjkdcSjCGJuC')
pub_key = priv_key.get_public_key()
from_address = pub_key.get_address()

# Receiving address
to_address = P2pkhAddress('mreLpAzPWBtdwBC9NMEsBy1jkQ3phjy1Eh')

# UTXO used
utxo_txid = "af587d919993ea39da724073d9bac9ea1e39e7b55a45d2ce1cb5e4119bc4b345"
utxo_vout = 1  # Index of the UTXO

# tBTC amount in the UTXO and transaction fee (calculated in satoshis)
amount_to_send = int(0.00012 * 100000000)
fee = int(0.00001 * 100000000) 
total_utxo = int(0.00013682 * 100000000)  
change_amount = total_utxo - amount_to_send - fee  

### Main Program

In [ ]:
# Setup connection to the testnet
setup('testnet')

# Create the transaction input and output with the UTXO
txin = TxInput(utxo_txid, utxo_vout)
txout = TxOutput(amount_to_send, to_address.to_script_pub_key())

# Output for change
change_address = from_address
change_output = TxOutput(change_amount, change_address.to_script_pub_key())

# Create the transaction
tx = Transaction([txin], [txout, change_output])

# Sign the transaction with your private key
script_pub_key = from_address.to_script_pub_key()  
sig = priv_key.sign_input(tx, 0, script_pub_key)
txin.script_sig = Script([sig, pub_key.to_hex()])

# Review the transaction and prepare it for broadcasting
raw_tx = tx.serialize()
print("Raw transaction: ", raw_tx)

# Broadcast the transaction to the testnet via the Blockstream API
response = requests.post('https://blockstream.info/testnet/api/tx', data=raw_tx)

# Check the response
if response.status_code == 200:
    print('Transaction ID:', response.text)
else:
    print('Error:', response.text)

Raw transaction: 020000000145b3c49b11e4b51cced2455ab5e7391eeac9bad9734072da39ea9399917d58af010000006a47304402201f14538a448e55ab80f42edd67a93b32336a21b7763172ebf5e821161d49239202204ff0f0a8b1b438104d03079d31773ea2d90d1b25be28365ae356b92717e54969012103db1db7e0d759dd6f87abf8be0fcf3d0464f71d234b2abfba5a12c147d39a06a6ffffffff02e02e0000000000001976a9147a0f17f08359d9bff71ad97f1eb39b694ad67c0488acaa020000000000001976a9143b519bbc4dcc90d447c25e02aa4a544ab23a6b7188ac00000000 

Transaction ID: e1d6d4a1dd80be08c107fa81555c78203ce04955ef4e56c56e7a5960a7989d90

# Exercise 2

## Question 1

### Import

In [ ]:
from bitcoinutils.setup import setup
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script
from bitcoinutils.keys import PrivateKey, P2pkhAddress
import requests

### Params

In [ ]:

# My keys
priv_key = PrivateKey('cUYYNWUYYMUQncmo8curJihmDdNXbGAKKWpiCb1MdjkdcSjCGJuC')
pub_key = priv_key.get_public_key()

second_pub_key = PublicKey('02019662a808d4a0df7e8c1ee8b26646e59cfaa92ebd906bde14b4bda5113fa2a9')

# UTXO details
utxo_txid = "d7a19890e135fb6f230f035c86f8feef5be5dab8f4d9a7f8492105d509fdb4f0"
utxo_vout = 0  

# Recipient
amount_to_send = int(0.0001 * 100000000)  # 10000 satoshis (0.0001 BTC)
fee = int(0.00001 * 100000000)  # 1000 satoshis
total_utxo = int(0.00013682 * 100000000)  # 13682 satoshis (0.00013682 BTC)
change_amount = total_utxo - amount_to_send - fee  # Số tiền thừa (682 satoshis)

### Main Program

In [ ]:
setup('testnet')

multisig_script = Script([
    'OP_2', pub_key.to_hex(), second_pub_key.to_hex(), 'OP_2', 'OP_CHECKMULTISIG'
])

# Create transaction input output
txin = TxInput(utxo_txid, utxo_vout)
txout = TxOutput(amount_to_send, multisig_script)

change_address = pub_key.get_address()  # Địa chỉ P2PKH của bạn để nhận lại tiền thừa
change_output = TxOutput(change_amount, change_address.to_script_pub_key())

# Transaction
tx = Transaction([txin], [txout, change_output])

# Sign
script_pub_key = change_address.to_script_pub_key()  # Script công khai của địa chỉ P2PKH
sig = priv_key.sign_input(tx, 0, script_pub_key)
txin.script_sig = Script([sig, pub_key.to_hex()])

raw_tx = tx.serialize()
print("Raw transaction:", raw_tx)

# Broadcast
response = requests.post('https://blockstream.info/testnet/api/tx', data=raw_tx)

if response.status_code == 200:
    print('Transaction ID:', response.text)
else:
    print('Error:', response.text)

Raw transaction: 0200000001f0b4fd09d5052149f8a7d9f4b8dae55beffef8865c030f236ffb35e19098a1d7000000006a4730440220048bf48c0a6db50cfd40f35181823c78007f39fc4286c0567adaf2e30e474c0d02204b03d4ef3ecdaf7a4266d3785522d84a9f73682f46195424bfabc57f58791d96012103db1db7e0d759dd6f87abf8be0fcf3d0464f71d234b2abfba5a12c147d39a06a6ffffffff02102700000000000047522103db1db7e0d759dd6f87abf8be0fcf3d0464f71d234b2abfba5a12c147d39a06a62102019662a808d4a0df7e8c1ee8b26646e59cfaa92ebd906bde14b4bda5113fa2a952ae7a0a0000000000001976a9143b519bbc4dcc90d447c25e02aa4a544ab23a6b7188ac00000000

Transaction ID: d89e3faa2d928b4d37e847133cd427fce536b2f5811522ff18020a43b3c8c48a

## Question 2

### Import

In [ ]:
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2pkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script

### Params

In [ ]:
# My keys
my_priv_key = PrivateKey('cUYYNWUYYMUQncmo8curJihmDdNXbGAKKWpiCb1MdjkdcSjCGJuC')
my_pub_key = my_priv_key.get_public_key()

# Teacher's key 
teacher_pub_key = PublicKey('02019662a808d4a0df7e8c1ee8b26646e59cfaa92ebd906bde14b4bda5113fa2a9')

# UTXO from the multisignature output
utxo_txid = "89436ccb83078094ac54795f51da6cbd538acdf3e8c8087e8bc6418b40dcff9a"
utxo_vout = 0  

#Recipient
to_address = P2pkhAddress('msfTfNj6FicTNBShCJBhoxvhHoM794cKsZ')
amount_to_send = int(0.00017769 * 100000000)  
fee = 500  
total_amount = amount_to_send - fee  

### Main Program

In [ ]:
setup('testnet')

multisig_script = Script([
    'OP_2', my_pub_key.to_hex(), teacher_pub_key.to_hex(), 'OP_2', 'OP_CHECKMULTISIG'
])

# Multisig script
my_pub_key_hex = my_pub_key.to_hex()  
multisig_script = Script(['OP_2', my_pub_key_hex, teacher_pub_key_hex, 'OP_2', 'OP_CHECKMULTISIG'])

# Create transaction
txin = TxInput(utxo_txid, utxo_vout)
txout = TxOutput(total_amount, Script(['OP_DUP', 'OP_HASH160', to_address.to_hash160(), 'OP_EQUALVERIFY', 'OP_CHECKSIG']))
tx = Transaction([txin], [txout])

# Sign the transaction
sig = my_priv_key.sign_input(tx, 0, multisig_script)

# Script_sig for the multisig transaction
txin.script_sig = Script([sig, my_pub_key.to_hex()])

unsigned_tx_hex = tx.serialize()
signature_hex = sig 

print("Unsigned Transaction:", unsigned_tx_hex)
print("First Signature:", signature_hex)

Unsigned Transaction: 02000000019affdc408b41c68b7e08c8e8f3cd8a53bd6cda515f7954ac94800783cb6c4389000000006a473044022057574ef7f8bba13f4f019d24869519ea6445c89c5dac8e78e121886871b789ea02200667430340c33086832a88a1e676e0f21417baea09cb992dea307bda95a36192012103db1db7e0d759dd6f87abf8be0fcf3d0464f71d234b2abfba5a12c147d39a06a6ffffffff0175430000000000001976a914853d5a191fd2ab19ebbd9519a696e5f30f7d119788ac00000000

First Signature: 3044022057574ef7f8bba13f4f019d24869519ea6445c89c5dac8e78e121886871b789ea02200667430340c33086832a88a1e676e0f21417baea09cb992dea307bda95a3619201